<div>
<img src=https://www.institutedata.com/wp-content/uploads/2019/10/iod_h_tp_primary_c.svg width="300">
</div>

# Lab 3.1.3:
# *Database Programming with Python and SQLite*

<a name="demo"></a>
## Using SQLite

### Advantages of SQLite

- does not run on a separate server
- creates portable SQL databases saved in a single file
- databases are stored in a very efficient manner and allow fast querying
- ideal for small databases or databases that need to be copied across machines.
- prototyping applications (e.g. as an embedded database server in a Python program).



### The `sqlite3` Command Line Utility

- useful for basic SQL tasks and database maintenance
- for creating and dropping databases, it may be safer to use the command line than to roll the code into a Python program

1. Add the sqlite3 installation folder to your PATH environment variable.
2. Open a command window.
3. Navigate to your preferred working directory.
4. Start the sqlite3 command line utility and create a database called "ex1":

`$ sqlite3 ex1`

Output:  

`SQLite version 3.8.5 2014-05-29 12:36:14
Enter ".help" for usage hints.
sqlite>`

*Nb. If you leave out the database name (after `sqlite3`, above) a new, temporary database will be created; it will be destroyed upon exiting sqlite3.*  


Enter the following commands at the sqlite prompt (not shown):

`create table tbl1(one varchar(10), two smallint);
insert into tbl1 values('hello!',10);
insert into tbl1 values('goodbye', 20);
select * from tbl1;`

Output:  

`hello!|10
goodbye|20`

Enter `.help` at the sqlite3 command prompt. This lists the available "dot commands".

Some of the more useful dot commands include:

`  
.backup ?DB? FILE      Backup DB (default "main") to FILE.
.cd DIRECTORY          Change the working directory to DIRECTORY
.clone NEWDB           Clone data into NEWDB from the existing database
.databases             List names and files of attached databases
.dump ?TABLE? ...      Dump the database in an SQL text format.
.excel                 Display the output of next command in a spreadsheet
.exit                  Exit this program
.headers on|off        Turn display of headers on or off
.help                  Show this message
.import FILE TABLE     Import data from FILE into TABLE
.indexes ?TABLE?       Show names of all indexes.
.quit                  Exit this program
.read FILENAME         Execute SQL in FILENAME
.restore ?DB? FILE     Restore content of DB (default "main") from FILE
.save FILE             Write in-memory database into FILE
.tables ?TABLE?        List names of tables.`


### The `sqlite3` package

The easiest way to incorporate an SQL database into a Python application is by using the `sqlite3` package for [`Python 2.7`](https://docs.python.org/2.7/library/sqlite3.html) or [`Python 3.x`](https://docs.python.org/3/library/sqlite3.html).

Open a connection to an SQLite database file.  As before, if the file does not already exist it will automatically be created.

In [1]:
import pandas as pd
import sqlite3
sqlite_db = 'test_db.sqlite'
conn = sqlite3.connect(sqlite_db)
c = conn.cursor()

Commands can be executed by passing them as string arguments to the `execute` method of the cursor we just created for this database.

*Nb. If this script has never been run before, the following will raise an (inconsequential) error message:*

In [2]:
c.execute('DROP TABLE houses;')

# Save (commit) the changes:
conn.commit()

In [3]:
c.execute('CREATE TABLE houses (field1 INTEGER PRIMARY KEY, sqft INTEGER, bdrms INTEGER, age INTEGER, price INTEGER);')
conn.commit()

With the database saved the table should now be viewable using SQLite Manager.

#### Adding data

Since we're back in python, we can now use regular programming techniques in conjunction with the sqlite connection.  In particular, the cursor's `execute()` method supports value substitution using the `?` character, which makes adding multiple records a bit easier.  See the [docs](https://docs.python.org/2.7/library/sqlite3.html) for more details.

> c.execute(sql_command, values)

In [4]:
# A
last_sale = (None, 4000, 5, 22, 619000)

# Execute Command
c.execute('INSERT INTO houses VALUES (?,?,?,?,?)',last_sale)

# Remember to commit the changes
conn.commit()

Notice that in this syntax we use the python `None` value, rather than `NULL`, to trigger SQLite to auto-increment the Primary Key.

There is a related cursor method `executemany()` which takes an array of tuples and loops through them, substituting one tuple at a time.

> c.executemany(sql_command, values)

In [5]:
# A
recent_sales = [
  (None, 2390, 4, 34, 319000),
  (None, 1870, 3, 14, 289000),
  (None, 1505, 3, 90, 269000),
]

# Nb. The "?" parameters are placeholders for data that will map to the table columns during insertion;
# this is a security measure against SQL injection attacks:
#
c.executemany('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', recent_sales)

conn.commit()

Select all rows from houses

In [6]:
# A
query = 'Select * FROM houses'
c.execute(query)
c.fetchall()
# pd.DataFrame(c.fetchall())

[(1, 4000, 5, 22, 619000),
 (2, 2390, 4, 34, 319000),
 (3, 1870, 3, 14, 289000),
 (4, 1505, 3, 90, 269000)]




#### Adding data from a csv file
One way to populate the database from a file is to use `numpy.genfromtxt` to read the file into an array (converted to a list for easier handling), and then `INSERT` those records into the database.  The `genfromtxt` function has options including the output data type, handling of missing values, skipping of header and footer rows, columns to read, and more.

In [7]:
from numpy import genfromtxt

# import into nparray of ints, then convert to list of lists:
data = genfromtxt('../../DATA/housing-data.csv', dtype = 'i8',
                    delimiter = ',', skip_header = 1).tolist()

Suppose we need to put a placeholder in the first column for data that will be available later. Best practice is to insert the value `None`:

In [8]:
# prepend a None value to beginning of each sub-list:
for d in data:
    d.insert(0, None)

*Nb. This is why we converted the input array to a list. An array can only hold one type of data (integers in this case) so we could not have inserted `None` before we did this conversion.*

In [9]:
# check:
print(type(data))
print(data[0:3])

<class 'list'>
[[None, 2104, 3, 70, 399900], [None, 1600, 3, 28, 329900], [None, 2400, 3, 44, 369000]]


Now we can insert each list item as a row of fields in the database.

In [10]:
# loop through data, running an INSERT on each record (i.e. sublist):
for d in data:
    c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d)

conn.commit()

In this case, because we were inserting the same value for all records, so we could have simply used a 'None' in the numpy `insert` method at column 0.

In [11]:
import numpy as np
d1 = np.asarray([1200, 3, 15, 250000])
d1 = d1.tolist()
d1.insert(0, None)
d1
c.execute('INSERT INTO houses VALUES (?, ?, ?, ?, ?)', d1)
conn.commit()

#### Deleting Rows

The `DELETE FROM` statement can be used with a `WHERE` clause to specify rows to delete based on some criteria.

In [12]:
c.execute('DELETE FROM houses WHERE field1 IN (52, 53)')
conn.commit()

QUIZ: What would `DELETE FROM houses` do?

#### Filtering Rows

**1. Select Rows Where Bedrooms = 4**

In [13]:
# ANSWER

In [14]:
results = c.execute("SELECT * FROM houses WHERE bdrms = 4")
results.fetchall()

[(2, 2390, 4, 34, 319000),
 (9, 3000, 4, 75, 539900),
 (10, 1985, 4, 61, 299900),
 (15, 1940, 4, 7, 239999),
 (20, 2300, 4, 77, 449900),
 (23, 2609, 4, 5, 499998),
 (24, 3031, 4, 21, 599000),
 (28, 1962, 4, 53, 259900),
 (37, 2040, 4, 75, 314900),
 (39, 1811, 4, 24, 285900),
 (42, 2132, 4, 28, 345000),
 (43, 4215, 4, 66, 549000),
 (44, 2162, 4, 43, 287000),
 (47, 2567, 4, 57, 314000),
 (50, 1852, 4, 64, 299900)]

**2. The following query calculates the average floor area and price of each size of house (i.e. by number of bedrooms):**

In [15]:
# ANSWER
results = c.execute("SELECT bdrms, AVG(sqft) AS avg_sqft, AVG(price) AS avg_price FROM houses GROUP BY bdrms")
results.fetchall()

[(1, 1000.0, 169900.0),
 (2, 1496.5, 280866.6666666667),
 (3, 1827.5555555555557, 322892.51851851854),
 (4, 2399.733333333333, 373553.13333333336),
 (5, 4239.0, 659450.0)]

### Pandas connector

While databases provide many analytical capabilities, at some point we may need to pull data into Python for more flexible processing. Large, fixed operations would be more efficient in a database, but Pandas allows for interactive processing.

For example, if you want to aggregate nightly log-ins or sales for a report or dashboard, this would be a fixed operation on a large dataset. These computations would run more efficiently in the database system itself.

However, if we wanted to model the patterns of login behaviour or factors driving sales, then we would import the data to Python where we could use its simple interfaces to powerful analytic libraries.

In [16]:
import pandas as pd
from pandas.io import sql

Pandas can connect to most relational databases. In this demonstration, we will create and connect to a SQLite database.

### Writing data into a database

Data in Pandas can be loaded into a relational database.

If the data table is not too large, we can load all of it into a Pandas DataFrame:

In [17]:
# Nb. Use low_memory=False to ensure that type inference does not fail due to buffered processing of input:
data = pd.read_csv('../../DATA/housing-data.csv', low_memory = False)
data.head()

,sqft,bdrms,age,price
0,2104,3,70,399900
1,1600,3,28,329900
2,2400,3,44,369000
3,1416,2,49,232000
4,3000,4,75,539900


We can move data in the opposite direction -- from a DataFrame to a database -- using the `to_sql` command, similar to the `to_csv` command.

`to_sql` takes as arguments:
    - `name`, the table name to create
    - `con`, a connection to a database
    - `index`, whether to input the index column
    - `schema`, if we want to write a custom schema for the new table
    - `if_exists`, what to do if the table already exists. We can overwrite it, add to it, or fail

This copies our `data` DataFrame to a sqlite3 table called `houses_pandas`:

In [18]:
data.to_sql('houses_pandas', con = conn, if_exists = 'replace', index = False)

47

Run a query to get the average price of each house size from this table:

In [ ]:
#ANSWER

In [26]:
query = c.execute("SELECT bdrms, AVG(price) AS avg_price FROM houses_pandas GROUP BY bdrms;")

avg_prices = pd.DataFrame(query.fetchall())
avg_prices.head()

,0,1
0,1,169900.000000
1,2,280866.666667
2,3,326403.920000
3,4,377449.785714
4,5,699900.000000


## Discussion

**Scenarios for using Pandas with SQLite**

1. When would you want to use Pandas on a dataset before storing it in a database?

2. When would you want to use Pandas on a dataset retrieved from a database?

<a name="guided-practice"></a>


# Reference: SQL Syntax


#### SELECT Statement
Every query should start with `SELECT`.  `SELECT` is followed by the names of the columns in the output.

`SELECT` is always paired with `FROM`, and `FROM` identifies the table to retrieve data from.

```sql
SELECT
<columns>
FROM
<table>
```

`SELECT *` denotes returns *all* of the columns.

Housing Data example:
```sql
SELECT
sqft, bdrms
FROM houses_pandas;
```

**Check:** Write a query that returns the `sqft`, `bdrms` and `price`.

```sql
SELECT
sqft, bdrms, price
FROM houses_pandas;
```

#### WHERE Clause
`WHERE` is used to filter table to a specific criteria and follows the `FROM` clause.

```sql
SELECT
<columns>
FROM
<table>
WHERE
<condition>
```
Example:
```sql
SELECT
sqft, bdrms, age, price
FROM houses_pandas
WHERE bdrms = 2 and price < 250000;
```

The condition is effectively a row filter; rows that match the condition will be included in the rowset that is returned by the query.

**Check:** Write a query that returns the `sqft`, `bdrms`, `age` for houses older than 60 years.
>```sql
SELECT
sqft, bdrms, age
FROM houses_pandas
WHERE age > 60;
```

### AGGREGATIONS

Aggregations (or aggregate functions) are functions where the values of multiple rows are grouped together as input on certain criteria to form a single value of more significant meaning or measurement such as a set, a bag or a list.

Examples of aggregate functions:

- Average (i.e., arithmetic mean)
- Count
- Maximum
- Minimum
- Median
- Mode
- Sum

In SQL they are performed in a `SELECT` statement as follows.

```sql
SELECT COUNT(price)
FROM houses_pandas;
```

```sql
SELECT AVG(sqft), MIN(price), MAX(price)
FROM houses_pandas
WHERE bdrms = 2;
```

### Read Order Data
- P12-ListOfOrders
- P12-OrderBreakdown

#### 1. Read CSV into DataFrame

In [31]:
# Reading CSV to Dataframe
orders = pd.read_csv('../../DATA/P12-ListOfOrders.csv', encoding = 'utf-8', low_memory = False)
orders_break_down =  pd.read_csv('../../DATA/P12-OrderBreakdown.csv', encoding = 'utf-8', low_memory = False)

In [32]:
orders.head()

,Order ID,Order Date,Customer Name,Country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [33]:
orders_break_down.head()

,Order ID,Product Name,Discount,Sales,Quantity,Category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


#### 2. Replace Space with Underscore in Column Names

In [34]:
# ANSWER
orders.columns = [o.replace(' ', '_') for o in orders.columns.str.lower()]

In [35]:
# ANSWER
orders_break_down.columns = [o.replace(' ', '_') for o in orders_break_down.columns.str.lower()]

#### 3. Check DataTypes

In [36]:
# ANSWER
orders.dtypes

order_id         object
order_date       object
customer_name    object
country          object
dtype: object

In [37]:
# ANSWER
orders_break_down.dtypes

order_id         object
product_name     object
discount        float64
sales           float64
quantity          int64
category         object
dtype: object

#### 4. Save these two dataframes as a table in sqlite

In [38]:
# Establishing Local DB connection
db_connection = sqlite3.connect('../../DATA/eshop.db.sqlite')

# Save these two dataframes as a table in sqlite

In [39]:
orders.to_sql('orders', con = db_connection, if_exists = 'replace', index = False)
orders_break_down.to_sql('orders_break_down', con = db_connection, if_exists = 'replace', index = False)

8047

#### 5. Select Number of Orders for Each Customer

We can use the `sql` function in `pandas.io` to run SQL queries to the database with the `.read_sql()` method. Here's an example:

In [40]:
# Select first 10 rows of all columns in orders table

query = """
        SELECT *
        FROM orders
        LIMIT 10;
        """

orders_head_10 = sql.read_sql(query, con=db_connection)
orders_head_10

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France
5,ES-2011-5433855,2011-01-07,Dave Poirier,France
6,IT-2011-4546695,2011-01-08,Darren Powers,France
7,ES-2011-1138719,2011-01-11,Eric Murdock,Italy
8,ES-2011-1466305,2011-01-11,Mick Brown,Austria
9,ES-2011-4359424,2011-01-11,Dorothy Dickinson,Spain


In [ ]:
# Select Number of Distinct Orders for Each Customer

In [48]:
query = """
        SELECT customer_name, COUNT(order_id) AS orders
        FROM orders
        GROUP BY customer_name
        ORDER BY COUNT(order_id) DESC;"""

distinct_orders = sql.read_sql(query, con=db_connection)
distinct_orders

,customer_name,orders
0,John Grady,13
1,Joel Jenkins,12
2,Aaron Smayling,12
3,Yoseph Carroll,11
4,Sarah Brown,11
...,...,...
787,Brooke Gillingham,1
788,Barbara Fisher,1
789,Art Ferguson,1
790,Andy Reiter,1


#### 6. Select Number of Customers for Each Country

In [ ]:
# Select Number of Customers for Each Country

In [50]:
query = """
        SELECT country, COUNT(customer_name) AS customers
        FROM orders
        GROUP BY country
        ORDER BY COUNT(customer_name) DESC"""

country = sql.read_sql(query, con=db_connection)
country

,country,customers
0,France,991
1,Germany,806
2,United Kingdom,700
3,Italy,493
4,Spain,403
5,Netherlands,194
6,Austria,135
7,Sweden,100
8,Belgium,68
9,Ireland,50


#### 7.A Select discount, sales, quantity for Each Order from orders_break_down Table

In [ ]:
#  Select discount, sales, quantity for Each Order from orders_break_down Table

In [51]:
orders_break_down.head()

,order_id,product_name,discount,sales,quantity,category
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies
3,IT-2011-2942451,"Boston Markers, Easy-Erase",0.5,27.0,2,Office Supplies
4,IT-2011-2942451,"Eldon Folders, Single Width",0.5,17.0,2,Office Supplies


In [53]:
query = """
        SELECT quantity, discount, sales
        FROM orders_break_down"""

order_details = sql.read_sql(query, con=db_connection)
order_details

,quantity,discount,sales
0,3,0.5,45.0
1,7,0.0,854.0
2,3,0.0,140.0
3,2,0.5,27.0
4,2,0.5,17.0
...,...,...,...
8042,2,0.0,245.0
8043,2,0.5,30.0
8044,4,0.5,23.0
8045,3,0.5,108.0


#### 7.B Select discount, sales, quantity, total price for Each Order from orders_break_down Table

> Total Price = sales * quantity - discount

In [ ]:
# Select discount, sales, quantity, total price for Each Order Id from orders_break_down Table

In [54]:
query = """
        SELECT quantity, discount, sales, sales*quantity-discount AS total_price
        FROM orders_break_down"""

total_price = sql.read_sql(query, con=db_connection)
total_price

,quantity,discount,sales,total_price
0,3,0.5,45.0,134.5
1,7,0.0,854.0,5978.0
2,3,0.0,140.0,420.0
3,2,0.5,27.0,53.5
4,2,0.5,17.0,33.5
...,...,...,...,...
8042,2,0.0,245.0,490.0
8043,2,0.5,30.0,59.5
8044,4,0.5,23.0,91.5
8045,3,0.5,108.0,323.5


#### 7.C Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [ ]:
# Select All Orders from orders_break_down Table Where Total Price Greater Than 100

In [56]:
query = """
        SELECT *, quantity*sales-discount AS total_price
        FROM orders_break_down
        WHERE total_price > 100"""

over_100 = sql.read_sql(query, con=db_connection)
over_100

,order_id,product_name,discount,sales,quantity,category,total_price
0,IT-2011-3647632,"Enermax Note Cards, Premium",0.5,45.0,3,Office Supplies,134.5
1,ES-2011-4869686,"Dania Corner Shelving, Traditional",0.0,854.0,7,Furniture,5978.0
2,ES-2011-4939443,"Binney & Smith Sketch Pad, Easy-Erase",0.0,140.0,3,Office Supplies,420.0
3,ES-2011-3848439,"Binney & Smith Pencil Sharpener, Water Color",0.0,90.0,3,Office Supplies,270.0
4,ES-2011-3848439,"Sanford Canvas, Fluorescent",0.0,207.0,4,Office Supplies,828.0
...,...,...,...,...,...,...,...
6461,ES-2014-2815584,"Kleencut Shears, Serrated",0.0,261.0,6,Office Supplies,1566.0
6462,ES-2014-4785777,"Wilson Jones Index Tab, Economy",0.0,32.0,5,Office Supplies,160.0
6463,ES-2014-3458802,"Ikea Stackable Bookrack, Traditional",0.0,245.0,2,Furniture,490.0
6464,IT-2014-3715679,"StarTech Phone, Red",0.5,108.0,3,Technology,323.5


#### 8. Select All Customers And The Products They Have Bought

In [ ]:
# Select All Order Ids, Customers And The Products (product_name) They Have Bought

In [63]:
query = """
        SELECT orders.customer_name, GROUP_CONCAT(orders_break_down.product_name, ', ') AS products
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY customer_name"""

products = sql.read_sql(query, con=db_connection)
products

,customer_name,products
0,Aaron Bergman,"Apple Office Telephone, Cordless, Elite Box Cu..."
1,Aaron Hawkins,"Epson Inkjet, Wireless, Xerox Note Cards, Recy..."
2,Aaron Smayling,"Binney & Smith Pens, Fluorescent, Apple Signal..."
3,Adam Bellavance,"Sauder Classic Bookcase, Mobile, Panasonic Pri..."
4,Adam Hart,"Cardinal Binder Covers, Economy, Ibico Binder ..."
...,...,...
787,Xylona Preis,"Ikea Classic Bookcase, Mobile, Smead Shelving,..."
788,Yana Sorensen,"Binney & Smith Sketch Pad, Water Color, Enerma..."
789,Yoseph Carroll,"Hon Round Labels, 5000 Label Set, Smead Folder..."
790,Zuschuss Carroll,"BIC Canvas, Blue, Breville Microwave, Silver, ..."


#### 9.A Select Number of 'Furniture' Orders For Each Country

In [ ]:
# Select Number of 'Furniture' Orders For Each Country

In [69]:
query = """
        SELECT country, COUNT(orders_break_down.order_id) AS furniture_orders
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
        WHERE orders_break_down.category IS 'Furniture'
        GROUP BY orders.country
        ORDER BY furniture_orders DESC"""

count_furniture = sql.read_sql(query, con=db_connection)
count_furniture

,country,furniture_orders
0,France,299
1,Germany,264
2,United Kingdom,180
3,Italy,136
4,Spain,123
5,Netherlands,76
6,Austria,40
7,Sweden,36
8,Belgium,20
9,Finland,16


#### 9.B Select Number of 'Furniture' Orders For The Country Denmark

In [ ]:
# Select Number of 'Furniture' Orders For The Country Denmark

In [70]:
query = """
        SELECT country, COUNT(orders_break_down.order_id) AS furniture_orders
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id = orders_break_down.order_id
        WHERE orders_break_down.category IS 'Furniture' AND orders.country IS 'Denmark'
        GROUP BY orders.country
        ORDER BY furniture_orders DESC"""

count_furniture = sql.read_sql(query, con=db_connection)
count_furniture

,country,furniture_orders
0,Denmark,9


#### 10. Select Total Sales With Discount and Without Discount for Each Country

In [ ]:
# Select Total Sales With Discount (discount > 0) and Without Discount (discount = 0) for Each Country

In [71]:
orders.head()

,order_id,order_date,customer_name,country
0,IT-2011-3647632,2011-01-01,Eugene Moren,Sweden
1,ES-2011-4869686,2011-01-03,Dorothy Dickinson,United Kingdom
2,ES-2011-4939443,2011-01-04,Arthur Prichep,France
3,IT-2011-2942451,2011-01-04,Grant Thornton,United Kingdom
4,ES-2011-3848439,2011-01-05,Michael Granlund,France


In [75]:
query = """
        SELECT country, SUM(quantity*sales-discount) AS sales_discount, SUM(quantity*sales) AS sales_no_discount
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY country
        ORDER BY sales_discount DESC"""

sales = sql.read_sql(query, con=db_connection)
sales

,country,sales_discount,sales_no_discount
0,France,3108445.00,3108592.0
1,Germany,2411393.15,2411483.0
2,United Kingdom,2161854.80,2161947.0
3,Spain,1340805.90,1340832.0
4,Italy,1177798.40,1177921.0
5,Austria,391694.00,391694.0
6,Netherlands,379525.10,379714.0
7,Belgium,229829.00,229829.0
8,Sweden,168357.90,168461.0
9,Switzerland,141661.00,141661.0


#### 11.A Select Total Quantity, Total Sales for Each Country

In [ ]:
# Select Total Quantity, Total Sales for Each Country

In [81]:
query = """
        SELECT orders.country, SUM(orders_break_down.quantity) AS total_quantity, sum(orders_break_down.quantity*orders_break_down.sales-orders_break_down.discount) AS total_sales
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY country
        ORDER BY total_sales DESC
        """

sales_country = sql.read_sql(query, con=db_connection)
sales_country

,country,total_quantity,total_sales
0,France,7329,3108445.00
1,Germany,6179,2411393.15
2,United Kingdom,4917,2161854.80
3,Spain,2881,1340805.90
4,Italy,3612,1177798.40
5,Austria,973,391694.00
6,Netherlands,1526,379525.10
7,Belgium,532,229829.00
8,Sweden,753,168357.90
9,Switzerland,308,141661.00


#### 11.B Select Top 3 Countries and Quantities Based on Total Sales

In [ ]:
# Select Top 3 Countries and Quantities Based on Total Sales

In [82]:
query = """
        SELECT orders.country, SUM(orders_break_down.quantity) AS total_quantity, sum(orders_break_down.quantity*orders_break_down.sales-orders_break_down.discount) AS total_sales
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY country
        ORDER BY total_sales DESC
        LIMIT 3
        """

top_sales_country = sql.read_sql(query, con=db_connection)
top_sales_country

,country,total_quantity,total_sales
0,France,7329,3108445.00
1,Germany,6179,2411393.15
2,United Kingdom,4917,2161854.80


#### 11.C Select Bottom 3 Countries and Sales Based On Total Quantities

In [ ]:
# Select Bottom 3 Countries and Sales Based On Total Quantities

In [84]:
query = """
        SELECT orders.country, SUM(orders_break_down.quantity) AS total_quantity, sum(orders_break_down.quantity*orders_break_down.sales-orders_break_down.discount) AS total_sales
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY country
        ORDER BY total_quantity ASC
        LIMIT 3
        """

bot_sales_country = sql.read_sql(query, con=db_connection)
bot_sales_country

,country,total_quantity,total_sales
0,Finland,201,82001.0
1,Denmark,204,34586.5
2,Norway,261,99878.0


#### 12. Select Average Sales By Category For The Country 'France'

In [ ]:
# Select Average Sales By Category For The Country 'France'

In [89]:
query = """
        SELECT category, AVG(sales) AS sales_avg
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        WHERE orders.country IS 'France'
        GROUP BY orders_break_down.category
        ORDER BY sales_avg DESC
        """

france = sql.read_sql(query, con=db_connection)
france

,category,sales_avg
0,Technology,595.145078
1,Furniture,582.314381
2,Office Supplies,167.217709


#### 13. Select Country, Category and Average Sales Where Average Sales is The Highest

In [ ]:
# Select Country, Category and Average Sales Where Average Sales is The Highest

In [91]:
query = """
        SELECT country, category, AVG(sales) AS sales_avg
        FROM orders
        LEFT JOIN orders_break_down ON orders.order_id=orders_break_down.order_id
        GROUP BY country
        ORDER BY sales_avg DESC
        """

top_cat = sql.read_sql(query, con=db_connection)
top_cat

,country,category,sales_avg
0,Spain,Office Supplies,327.729304
1,Finland,Office Supplies,323.468750
2,United Kingdom,Furniture,320.012938
3,Switzerland,Office Supplies,318.897436
4,France,Office Supplies,318.206159
5,Belgium,Office Supplies,313.481481
6,Austria,Office Supplies,300.689394
7,Germany,Office Supplies,297.976220
8,Norway,Office Supplies,293.271429
9,Italy,Office Supplies,258.163432


### JOINS

Below is a link to a handy reference for SQL joins. In this chart joins are represented in terms of sets and venn diagrams.
https://www.codeproject.com/Articles/33052/Visual-Representation-of-SQL-Joins

Alternatively, remember the merge functionality of pandas.
https://github.com/pandas-dev/pandas/blob/master/doc/cheatsheet/Pandas_Cheat_Sheet.pdf

### ADDITIONAL RESOURCES

- [sqlite3 home](http://www.sqlite.org)  
- [sqlite3 Python documentation](https://docs.python.org/3/library/sqlite3.html)
- [SQLite Python tutorial](http://sebastianraschka.com/Articles/2014_sqlite_in_python_tutorial.html)  



---



---



> > > > > > > > > © 2023 Institute of Data


---



---



